# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
val housing = spark.read.format("csv")
                        .option("inferSchema", "true")
                        .option("header", "true")
                        .load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
housing.printSchema

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [3]:
housing.count

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [4]:
housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [5]:
housing.where('population > 10000).count

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [6]:
housing.describe("housing_median_age", "total_rooms","median_house_value", "population").show

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [7]:
import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age")).show

housing.agg(min("total_rooms")).show

housing.agg(avg("median_house_value")).show

+-----------------------+
|max(housing_median_age)|
+-----------------------+
|                   52.0|
+-----------------------+

+----------------+
|min(total_rooms)|
+----------------+
|             2.0|
+----------------+

+-----------------------+
|avg(median_house_value)|
+-----------------------+
|     206855.81690891474|
+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [8]:
housing.groupBy("ocean_proximity").count.orderBy(desc("count")).show

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [9]:
housing.groupBy("ocean_proximity").agg(avg("median_house_value").alias("avg_value")).show

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [10]:
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [11]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

va = vecAssembler_570e339acabf
housingAttrs = [longitude: double, latitude: double ... 9 more fields]


[longitude: double, latitude: double ... 9 more fields]

In [12]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


coeff = 


1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [13]:
val housingCol1 = housing.withColumn("rooms_per_household", 'total_rooms / 'households)
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", 'total_bedrooms / 'total_rooms)
val housingExtra = housingCol2.withColumn("population_per_household", 'population / 'households)

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1 = [longitude: double, latitude: double ... 9 more fields]
housingCol2 = [longitude: double, latitude: double ... 10 more fields]
housingExtra = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [14]:
val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")
renamedHousing.printSchema

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- label: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- rooms_per_household: double (nullable = true)
 |-- bedrooms_per_room: double (nullable = true)
 |-- population_per_household: double (nullable = true)



renamedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [15]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel = label
colCat = ocean_proximity
colNum = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [16]:
for (c <- colNum) {
    print(s"Nulls in column $c: ")
    println(renamedHousing.where(col(c).isNull).count)
}

Nulls in column longitude: 0
Nulls in column latitude: 0
Nulls in column housing_median_age: 0
Nulls in column total_rooms: 0
Nulls in column total_bedrooms: 207
Nulls in column population: 0
Nulls in column households: 0
Nulls in column median_income: 0
Nulls in column rooms_per_household: 0
Nulls in column bedrooms_per_room: 207
Nulls in column population_per_household: 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [17]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms", "bedrooms_per_room")).setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

for (c <- colNum) {
    print(s"Nulls in column $c: ")
    println(imputedHousing.where(col(c).isNull).count)
}

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

Nulls in column longitude: 0
Nulls in column latitude: 0
Nulls in column housing_median_age: 0
Nulls in column total_rooms: 0
Nulls in column total_bedrooms: 0
Nulls in column population: 0
Nulls in column households: 0
Nulls in column median_income: 0
Nulls in column rooms_per_household: 0
Nulls in column bedrooms_per_room: 0
Nulls in column population_per_household: 0


imputer = imputer_89b848e22ec3
imputedHousing = [longitude: double, latitude: double ... 11 more fields]


[longitude: double, latitude: double ... 11 more fields]

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vector of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [18]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("imputed_features")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("imputed_features").setOutputCol("rescaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|    imputed_features|   rescaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

va = vecAssembler_797819decf94
featuredHousing = [longitude: double, latitude: double ... 12 more fields]
scaler = stdScal_b52e573ff477
scaledHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [19]:
renamedHousing.select("ocean_proximity").distinct.show

+---------------+
|ocean_proximity|
+---------------+
|         ISLAND|
|     NEAR OCEAN|
|       NEAR BAY|
|      <1H OCEAN|
|         INLAND|
+---------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [20]:
import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_num")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_num|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0

indexer = strIdx_597ac8353243
idxHousing = [longitude: double, latitude: double ... 12 more fields]


[longitude: double, latitude: double ... 12 more fields]

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [21]:
indexer.fit(renamedHousing).labels

Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [22]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("ocean_proximity_num")).setOutputCols(Array("ocean_proximity_oh"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_num|ocean_proximity_oh|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------+------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                3.0|     (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    24

encoder = oneHotEncoder_4828639da585
ohHousing = [longitude: double, latitude: double ... 13 more fields]


[longitude: double, latitude: double ... 13 more fields]

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [23]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-------------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|    imputed_features|   rescaled_features|ocean_proximity_num|ocean_proximity_oh|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-------------------+------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.555

numPipeline = pipeline_e32e956bd523
catPipeline = pipeline_7b384480afe9
pipeline = pipeline_702ef6496a19
newHousing = [longitude: double, latitude: double ... 15 more fields]


[longitude: double, latitude: double ... 15 more fields]

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [24]:
// val newCols = Array("longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "label", "rooms_per_household", "bedrooms_per_room", "population_per_household", "imputed_features", "rescaled_features", "ocean_proximity_num", "ocean_proximity_oh")
val newCols = Array("rescaled_features", "ocean_proximity_oh")
// val newCols = Array("rescaled_features")

val va2 = new VectorAssembler().setInputCols(newCols).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



newCols = Array(rescaled_features, ocean_proximity_oh)
va2 = vecAssembler_1b5819739729
dataset = [features: vector, label: double]


[features: vector, label: double]

In [25]:
// dataset.select("features").collect().foreach(println)

Name: Syntax Error.
Message: 
StackTrace: 

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [26]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(80,20))

trainSet = [features: vector, label: double]
testSet = [features: vector, label: double]


[features: vector, label: double]

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [27]:
import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression
// lr.setFeaturesCol("features")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: " + lrModel.coefficients + s", Intercept: " + lrModel.intercept)
println(s"RMSE: " + trainingSummary.rootMeanSquaredError)

Coefficients: [-55048.68235098975,-55238.58902963683,13738.518383813913,6385.804562735405,5830.469596074306,-46178.15127142237,39233.42006493015,78362.68783881779,6026.181673735514,15937.402062698338,4356.582703561229,-118689.44504884348,-154324.3452777393,-115576.28818636642,-122963.3793355763], Intercept: -2299230.395348986
RMSE: 67839.53454458452


lr = linReg_10b855297717
lrModel = linReg_10b855297717
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@29150672


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@29150672

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [28]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 208413.8862537546|106700.0|[-62.005506847089...|
|108783.62672471255| 66900.0|[-61.995524474579...|
| 121772.9868140188| 68400.0|[-61.995524474579...|
|155016.02216718253| 72200.0|[-61.980550915813...|
|148908.76076218905| 67000.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68592.55538110771


predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_d589d4217b9a
rmse = 68592.55538110771


68592.55538110771

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [29]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|167799.77239018824|106700.0|[-62.005506847089...|
|138060.93997965413| 66900.0|[-61.995524474579...|
|167799.77239018824| 68400.0|[-61.995524474579...|
|167799.77239018824| 72200.0|[-61.980550915813...|
|138060.93997965413| 67000.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68257.96448638712


dt = dtr_24f10fefbce7
dtModel = DecisionTreeRegressionModel (uid=dtr_24f10fefbce7) of depth 5 with 63 nodes
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_a6a6334e3841
rmse = 68257.96448638712


68257.96448638712

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [30]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|215394.97470044895|106700.0|[-62.005506847089...|
| 150167.3782746896| 66900.0|[-61.995524474579...|
| 157818.6380156837| 68400.0|[-61.995524474579...|
|153562.94952143717| 72200.0|[-61.980550915813...|
| 149698.9497798939| 67000.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 65153.704485503236


rf = rfr_6648ce2b852e
rfModel = RandomForestRegressionModel (uid=rfr_6648ce2b852e) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_0b4ec88f0248
rmse = 65153.704485503236


65153.704485503236

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [31]:
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|119174.64435783583|106700.0|[-62.005506847089...|
|101332.64419157246| 66900.0|[-61.995524474579...|
| 131234.1550906458| 68400.0|[-61.995524474579...|
| 85805.85035805378| 72200.0|[-61.980550915813...|
| 89650.82897437165| 67000.0|[-61.980550915813...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 56431.808841646154


gb = gbtr_bb198604f948
gbModel = GBTRegressionModel (uid=gbtr_bb198604f948) with 20 trees
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_538bec52ed68
rmse = 56431.808841646154


56431.808841646154

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [32]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
    .addGrid(rfModel.numTrees, Array(1,  5, 10))
    .addGrid(rfModel.maxDepth, Array(5, 10, 15))
    .build()

val evaluator = new RegressionEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("rmse")

val cv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(dataset)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+-----------------+--------+--------------------+
|       prediction|   label|            features|
+-----------------+--------+--------------------+
|         110352.0|106700.0|[-62.005506847089...|
|80810.04347826086| 66900.0|[-61.995524474579...|
|97170.04347826086| 68400.0|[-61.995524474579...|
| 80857.7628920787| 72200.0|[-61.980550915813...|
|81558.62364243944| 67000.0|[-61.980550915813...|
+-----------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 29397.543942137407


paramGrid = 


Array({
	rfr_6648ce2b852e-maxDepth: 5,
	rfr_6648ce2b852e-numTrees: 1
}, {
	rfr_6648ce2b852e-maxDepth: 5,
	rfr_6648ce2b852e-numTrees: 5
}, {
	rfr_6648ce2b852e-maxDepth: 5,
	rfr_6648ce2b852e-numTrees: 10
}, {
	rfr_6648ce2b852e-maxDepth: 10,
	rfr_6648ce2b852e-numTrees: 1
}, {
	rfr_6648ce2b852e-maxDepth: 10,
	rfr_6648ce2b852e-numTrees: 5
}, {
	rfr_6648ce2b852e-maxDepth: 10,
	rfr_6648ce2b852e-numTrees: 10
}, {
	rfr_6648ce2b852e-maxDepth: 15,
	rfr_6648ce2b852e-numTrees: 1
}, {
	rfr_6648ce2b852e-maxDepth: 15,
	rfr_6648ce2b852e-numTrees: 5
}, {
	rfr_6648ce2b852e-maxDepth: 15,
	rfr_6648...


---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [33]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "The first input column")
    final val inputCol2 = new Param[String](this, "inputCol2", "The second input column")
    final val outputCol = new Param[String](this, "outputCol", "The output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    // Check that the input type is a double
    val idx1 = schema.fieldIndex($(inputCol1))
    val idx2 = schema.fieldIndex($(inputCol2))
    val field1 = schema.fields(idx1)
    val field2 = schema.fields(idx2)
    if (field1.dataType != DoubleType) {
        throw new Exception(s"Input1 type ${field1.dataType} did not match input type DoubleType")
    }
    if (field2.dataType != DoubleType) {
        throw new Exception(s"Input2 type ${field2.dataType} did not match input type DoubleType")
    }
    // Add the return field
    schema.add(StructField($(outputCol), DoubleType, false))
  }
}


defined trait MyParams


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [34]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}

import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.types.StructField

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    }

    override def transformSchema(schema: StructType): StructType = {
        // Add the return field
        schema.add(StructField($(outputCol), DoubleType, false))
      }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        dataset.withColumn($(outputCol), col($(inputCol1)) / col($(inputCol2)))
  }
}

defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [35]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

+-------------------+
|rooms_per_household|
+-------------------+
|  6.984126984126984|
|  6.238137082601054|
|  8.288135593220339|
| 5.8173515981735155|
|  6.281853281853282|
+-------------------+
only showing top 5 rows



myTransformer = configurablewordcount_a2572a0d88d4


myDataset: Unit = ()


configurablewordcount_a2572a0d88d4

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [36]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

defined class Instance
defined object Helper


In [37]:
class LR() extends Serializable {
    
    
// https://stackoverflow.com/questions/44450889/why-does-spark-shell-fail-to-load-a-file-with-class-with-rdd-imported
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
    
    
    
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

defined class LR


In [38]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends org.apache.spark.ml.PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: org.apache.spark.ml.linalg.Vector, val trainingError: Array[Double])
    extends MyLinearModel[org.apache.spark.ml.linalg.Vector, MyLinearModelImpl] {

  override def copy(extra: org.apache.spark.ml.param.ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: org.apache.spark.ml.linalg.Vector): Double = {
//     println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

defined class MyLinearModel
defined class MyLinearModelImpl


In [39]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

defined class MyLinearRegression
defined class MyLinearRegressionImpl


In [40]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Training
+--------------------+--------+--------------------+
|          prediction|   label|            features|
+--------------------+--------+--------------------+
|-8.65598910280389...| 94600.0|[-62.065401082150...|
|-8.66755410195001...| 85800.0|[-62.040445150874...|
|-8.66884831433994E37|103600.0|[-62.040445150874...|
|-8.65549752452394...| 79000.0|[-62.025471592109...|
|-8.65180908403000...|111400.0|[-62.020480405854...|
+--------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 8.274544247662991E37


lr = linReg_d5b093cd74f3
model = linReg_d5b093cd74f3
predictions = [features: vector, label: double ... 1 more field]
evaluator = regEval_363265eed41a
rmse = 8.274544247662991E37


8.274544247662991E37

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

Col `PAY_0` should be renamed to `PAY_1`.

In [41]:
val cc = spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("data/ccdefault.csv")
    .withColumnRenamed("PAY_0", "PAY_1")

cc.cache()

cc = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [42]:
cc.printSchema

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_1: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [43]:
cc.count

30000

In [44]:
cc.show

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

In [45]:
cc.describe("LIMIT_BAL", "AGE", "PAY_1", "BILL_AMT1", "PAY_AMT1","DEFAULT").show
// cc.describe(cc.columns:_*).show

+-------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|summary|         LIMIT_BAL|              AGE|             PAY_1|        BILL_AMT1|         PAY_AMT1|            DEFAULT|
+-------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|  count|             30000|            30000|             30000|            30000|            30000|              30000|
|   mean|167484.32266666667|          35.4855|           -0.0167|       51223.3309|        5663.5805|             0.2212|
| stddev|129747.66156720246|9.217904068090155|1.1238015279973335|73635.86057552966|16563.28035402577|0.41506180569093254|
|    min|             10000|               21|                -2|          -165580|                0|                  0|
|    max|           1000000|               79|                 8|           964511|           873552|                  1|
+-------+---------------

In [46]:
for (c <- cc.columns) {
    print(s"Number of unique values in column $c: ")
//     cc.select(col(c)).distinct.show
    println(cc.select(col(c)).distinct.count)
}

Number of unique values in column ID: 30000
Number of unique values in column LIMIT_BAL: 81
Number of unique values in column SEX: 2
Number of unique values in column EDUCATION: 7
Number of unique values in column MARRIAGE: 4
Number of unique values in column AGE: 56
Number of unique values in column PAY_1: 11
Number of unique values in column PAY_2: 11
Number of unique values in column PAY_3: 11
Number of unique values in column PAY_4: 11
Number of unique values in column PAY_5: 10
Number of unique values in column PAY_6: 10
Number of unique values in column BILL_AMT1: 22723
Number of unique values in column BILL_AMT2: 22346
Number of unique values in column BILL_AMT3: 22026
Number of unique values in column BILL_AMT4: 21548
Number of unique values in column BILL_AMT5: 21010
Number of unique values in column BILL_AMT6: 20604
Number of unique values in column PAY_AMT1: 7943
Number of unique values in column PAY_AMT2: 7899
Number of unique values in column PAY_AMT3: 7518
Number of uniqu

In [47]:
for (c <- cc.columns) {
    print(s"Nulls in column $c: ")
    println(cc.where(col(c).isNull).count)
}

Nulls in column ID: 0
Nulls in column LIMIT_BAL: 0
Nulls in column SEX: 0
Nulls in column EDUCATION: 0
Nulls in column MARRIAGE: 0
Nulls in column AGE: 0
Nulls in column PAY_1: 0
Nulls in column PAY_2: 0
Nulls in column PAY_3: 0
Nulls in column PAY_4: 0
Nulls in column PAY_5: 0
Nulls in column PAY_6: 0
Nulls in column BILL_AMT1: 0
Nulls in column BILL_AMT2: 0
Nulls in column BILL_AMT3: 0
Nulls in column BILL_AMT4: 0
Nulls in column BILL_AMT5: 0
Nulls in column BILL_AMT6: 0
Nulls in column PAY_AMT1: 0
Nulls in column PAY_AMT2: 0
Nulls in column PAY_AMT3: 0
Nulls in column PAY_AMT4: 0
Nulls in column PAY_AMT5: 0
Nulls in column PAY_AMT6: 0
Nulls in column DEFAULT: 0


### Prepare one hot encoding

After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute ocean_proximity.

In [48]:
// import org.apache.spark.ml.feature.IntegerIndexer

// val indexer = new IntegerIndexer()
//     .setInputCol("SEX")
//     .setOutputCol("SEX_ind")

// val ohHousing = indexer.fit(cc).transform(cc)

// ohHousing.show(5)

Name: Syntax Error.
Message: 
StackTrace: 

In [49]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("AGE")).setOutputCols(Array("AGE_oh"))
val cc_oh = encoder.fit(cc).transform(cc)

cc_oh.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+---------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|         AGE_oh|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+---------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|(79,[24],[1.0])|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|    

encoder = oneHotEncoder_e4f19156179b
cc_oh = [ID: int, LIMIT_BAL: int ... 24 more fields]


[ID: int, LIMIT_BAL: int ... 24 more fields]

### Vector Assembler

In [50]:
import org.apache.spark.ml.feature.VectorAssembler

// feature columns
val featCols = cc.columns.filter(_ !="DEFAULT").filter(_ !="ID")

println(featCols)

val va = new VectorAssembler().setInputCols(featCols).setOutputCol("features")

val ccAttrs = va.transform(cc).withColumnRenamed("DEFAULT", "label")

ccAttrs.select("features").show(5)

[Ljava.lang.String;@5db2775d
+--------------------+
|            features|
+--------------------+
|[20000.0,2.0,2.0,...|
|[120000.0,2.0,2.0...|
|[90000.0,2.0,2.0,...|
|[50000.0,2.0,2.0,...|
|[50000.0,1.0,2.0,...|
+--------------------+
only showing top 5 rows



featCols = Array(LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
va = vecAssembler_9265e1076ff8
ccAttrs = [ID: int, LIMIT_BAL: int ... 24 more fields]


[ID: int, LIMIT_BAL: int ... 24 more fields]

In [51]:
// ccAttrs.select("features").collect().foreach(println)

Name: Syntax Error.
Message: 
StackTrace: 

## Models

In [52]:
val Array(trainSet, testSet) = ccAttrs.randomSplit(Array(80,20))
// trainSet.printSchema

trainSet = [ID: int, LIMIT_BAL: int ... 24 more fields]
testSet = [ID: int, LIMIT_BAL: int ... 24 more fields]


[ID: int, LIMIT_BAL: int ... 24 more fields]

### Logistic Regression

In [53]:
import org.apache.spark.ml.classification.LogisticRegression

// train the model
val lr = new LogisticRegression
// lr.setFeaturesCol("features")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: " + lrModel.coefficients + s", Intercept: " + lrModel.intercept)
// println(s"RMSE: " + trainingSummary.rootMeanSquaredError)

Coefficients: [-7.440186577505924E-7,-0.07566708402173794,-0.07587947875370081,-0.1300130385566917,0.008100942296413373,0.558342717316102,0.09306062386547195,0.08269913014853532,-0.005842930608991294,0.03583566471178993,0.023518145134792657,-3.621125033895038E-6,3.259066570794538E-7,2.654501073838579E-7,1.312017371044668E-6,8.013124303130368E-7,1.364427524257896E-7,-1.154357242889569E-5,-9.849875125797534E-6,-4.3459601031818E-6,-4.853964361038283E-6,-4.157694131737553E-6,-2.195505305489278E-6], Intercept: -0.8369221020629418


lr = logreg_cfdb85bc9853
lrModel = logreg_cfdb85bc9853
trainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@40917cd0


org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@40917cd0

In [54]:
val trainingSummary = lrModel.binarySummary
// obtain the objective per iteration.
val objectiveHistory = trainingSummary.objectiveHistory
// objectiveHistory.foreach(loss => println(loss))
// obtain the ROC as a dataframe and areaUnderROC.
val roc = trainingSummary.roc
roc.show()
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")
// set the model threshold to maximize F-Measure
val fMeasure = trainingSummary.fMeasureByThreshold
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where($"F-Measure" === maxFMeasure)
    .select("threshold").head().getDouble(0)
lrModel.setThreshold(bestThreshold)

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.003467221422094...|0.03272863287824949|
|0.007254494052381715|0.06470918271928185|
|0.010348322398250386| 0.0987469609126613|
|0.013975569424441244|0.13091453151299795|
|0.018029551394889848| 0.1615859360389003|
|0.021336747212887395|0.19487563119506265|
|   0.025390729183336|0.22554703572096502|
|0.029924787966074572| 0.2545352534131289|
| 0.03483223982503867| 0.2822143257901627|
| 0.04059316157251827|0.30690106601832806|
| 0.04560729716754681|0.33420609687675334|
|  0.0513682189150264|0.35889283710491865|
| 0.05835600362724703|0.37927809986908545|
| 0.06539713020749986|  0.399476341873948|
| 0.07307835920413933|0.41743033476715913|
|  0.0813463487491332| 0.4333270993080232|
| 0.08988104763428816| 0.4484757808116701|
| 0.09889582333173308|0.46212829624088275|
| 0.10839067584146797|0.47391060407705254|
+----------

trainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@40917cd0
objectiveHistory = Array(0.529330420970103, 0.48529686761155055, 0.4814768784165698, 0.4806266503513184, 0.47593009518495577, 0.47218487575854184, 0.46849826039953024, 0.4682960984891481, 0.46776842800493623, 0.46768750492984473, 0.467547737438641, 0.46744853388469587, 0.46734383344746655, 0.467292200328124, 0.4672871216653082, 0.46727746891990746, 0.4672674435144481, 0.46725514805557394, 0.4672315577839583, 0.4672054380098168, 0.4671762541732288, 0.46715612012422425, 0.46714649619824655, 0.4671290669536386, 0.4671212646301025, 0.4671138824306007, 0.4671125199111428, 0.46710958627160126, 0.467...


Array(0.529330420970103, 0.48529686761155055, 0.4814768784165698, 0.4806266503513184, 0.47593009518495577, 0.47218487575854184, 0.46849826039953024, 0.4682960984891481, 0.46776842800493623, 0.46768750492984473, 0.467547737438641, 0.46744853388469587, 0.46734383344746655, 0.467292200328124, 0.4672871216653082, 0.46727746891990746, 0.4672674435144481, 0.46725514805557394, 0.4672315577839583, 0.4672054380098168, 0.4671762541732288, 0.46715612012422425, 0.46714649619824655, 0.4671290669536386, 0.4671212646301025, 0.4671138824306007, 0.4671125199111428, 0.46710958627160126, 0.467...

In [55]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val lrParamGrid = new ParamGridBuilder()
    .addGrid(lrModel.regParam, Array(0.01, 0.1, 1))
    .build()

val lrEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val lrCv = new CrossValidator()
    .setEstimator(lr)
    .setEvaluator(lrEvaluator)
    .setEstimatorParamMaps(lrParamGrid)
    .setNumFolds(3)

val lrCvModel = lrCv.fit(ccAttrs)

val lrPredictions = lrCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val lrAuc = lrEvaluator.evaluate(lrPredictions)
println(s"Area Under ROC (AUC) on test data = $lrAuc")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|[500000.0,1.0,1.0...|
|       0.0|    0|[100000.0,2.0,2.0...|
|       0.0|    0|[630000.0,2.0,2.0...|
|       0.0|    0|[50000.0,2.0,3.0,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.7360441300936471


lrParamGrid = 
lrEvaluator = binEval_3d79d5dbef8d
lrCv = cv_d63dda347c90
lrCvModel = cv_d63dda347c90
lrPredictions = [ID: int, LIMIT_BAL: int ... 27 more fields]
lrAuc = 0.7360441300936471


Array({
	logreg_cfdb85bc9853-regParam: 0.01
}, {
	logreg_cfdb85bc9853-regParam: 0.1
}, {
	logreg_cfdb85bc9853-regParam: 1.0
})


0.7360441300936471

### Decision Tree

In [56]:
import org.apache.spark.ml.classification.DecisionTreeClassifier

val dt = new DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val dtPredictions = dtModel.transform(testSet)
dtPredictions.select("prediction", "rawPrediction", "label", "features").show(5)
// predictions.printSchema

+----------+--------------+-----+--------------------+
|prediction| rawPrediction|label|            features|
+----------+--------------+-----+--------------------+
|       0.0|[7043.0,647.0]|    0|[500000.0,1.0,1.0...|
|       0.0|[2202.0,429.0]|    0|[100000.0,2.0,2.0...|
|       0.0|[7043.0,647.0]|    0|[630000.0,2.0,2.0...|
|       0.0| [260.0,158.0]|    0|[50000.0,2.0,3.0,...|
|       0.0| [879.0,402.0]|    0|(23,[0,1,2,3,4,5,...|
+----------+--------------+-----+--------------------+
only showing top 5 rows



dt = dtc_fdae2b3a7026
dtModel = DecisionTreeClassificationModel (uid=dtc_fdae2b3a7026) of depth 5 with 63 nodes
dtPredictions = [ID: int, LIMIT_BAL: int ... 27 more fields]


[ID: int, LIMIT_BAL: int ... 27 more fields]

In [57]:
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val dtPredictionAndLabels = dtPredictions.select("rawPrediction", "label").rdd.map(x => (x(0).asInstanceOf[DenseVector](1), x(1).toString.toDouble))
val dtMetrics = new BinaryClassificationMetrics(dtPredictionAndLabels)

dtMetrics.areaUnderROC

dtPredictionAndLabels = MapPartitionsRDD[4106] at map at <console>:108
dtMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@40dbea39


0.40074568418767403

In [58]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

// DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
//             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
//             min_samples_split=42, min_weight_fraction_leaf=0.0,
//             presort=False, random_state=0, splitter='best')
// # Use a grid over parameters of interest
// param_grid = { 
//            "n_estimators" : [9, 18, 27, 36, 45, 54, 63],
//            "max_depth" : [1, 5, 10, 15, 20, 25, 30],
//            "min_samples_leaf" : [1, 2, 4, 6, 8, 10]}

val dtParamGrid = new ParamGridBuilder()
    .addGrid(dtModel.maxDepth, Array(1, 2, 4, 8, 16, 30))
    .addGrid(dtModel.maxBins, Array(2, 4, 8, 16, 32))
    .build()

val dtEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val dtCv = new CrossValidator()
    .setEstimator(dt)
    .setEvaluator(dtEvaluator)
    .setEstimatorParamMaps(dtParamGrid)
    .setNumFolds(3)

val dtCvModel = dtCv.fit(ccAttrs)

val dtPredictions = dtCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val dtAuc = dtEvaluator.evaluate(dtPredictions)
println(s"Area Under ROC (AUC) on test data = $dtAuc")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|[500000.0,1.0,1.0...|
|       0.0|    0|[100000.0,2.0,2.0...|
|       0.0|    0|[630000.0,2.0,2.0...|
|       0.0|    0|[50000.0,2.0,3.0,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

Area Under ROC (AUC) on test data = 0.9753781224412159


dtParamGrid = 


Array({
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 1
}, {
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 2
}, {
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 4
}, {
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 8
}, {
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 16
}, {
	dtc_fdae2b3a7026-maxBins: 2,
	dtc_fdae2b3a7026-maxDepth: 30
}, {
	dtc_fdae2b3a7026-maxBins: 4,
	dtc_fdae2b3a7026-maxDepth: 1
}, {
	dtc_fdae2b3a7026-maxBins: 4,
	dtc_fdae2b3a7026-maxDepth: 2
}, {
	dtc_fdae2b3a7026-maxBins: 4,
	dtc_fdae2b3...


### Random Forest

In [59]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

val rf = new RandomForestClassifier().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val rfPredictions = rfModel.transform(testSet)
rfPredictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|[500000.0,1.0,1.0...|
|       0.0|    0|[100000.0,2.0,2.0...|
|       0.0|    0|[630000.0,2.0,2.0...|
|       0.0|    0|[50000.0,2.0,3.0,...|
|       0.0|    0|(23,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows



rf = rfc_dfbc73461fca
rfModel = RandomForestClassificationModel (uid=rfc_dfbc73461fca) with 20 trees
rfPredictions = [ID: int, LIMIT_BAL: int ... 27 more fields]


[ID: int, LIMIT_BAL: int ... 27 more fields]

In [60]:
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val rfPredictionAndLabels = rfPredictions.select("rawPrediction", "label").rdd.map(x => (x(0).asInstanceOf[DenseVector](1), x(1).toString.toDouble))
val rfMetrics = new BinaryClassificationMetrics(rfPredictionAndLabels)

rfMetrics.areaUnderROC

rfPredictionAndLabels = MapPartitionsRDD[10891] at map at <console>:114
rfMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@3988459e


0.7763695171132807

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val rfParamGrid = new ParamGridBuilder()
    .addGrid(rfModel.maxDepth, Array(1, 2, 4, 8, 16, 30))
    .addGrid(rfModel.maxBins, Array(2, 4, 8, 16, 32))
    .build()

val rfEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val rfCv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(rfEvaluator)
    .setEstimatorParamMaps(rfParamGrid)
    .setNumFolds(3)

val rfCvModel = rfCv.fit(ccAttrs)

val rfPredictions = rfCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val rfAuc = rfEvaluator.evaluate(rfPredictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")